# Extract, Transform, Load (ETL)
---
* Source `https://www.eia.gov/opendata/browser/`
* Main Route `Electricity`
    * Sub Route 1 - `Electric Power Operations (Annual And Monthly)`    
    * Sub Route 2 - `State Specific Data`
        * Sub Sub Route - `Emissions From Energy Consumption At Conventional Power Plants and Combined-Heat-And-Power Plants`
* Year range `2012` to `2022`
---
API Key can be obtained via signing up via `https://www.eia.gov/opendata/` then added to to the variable `api_key` located under the `config.py` file. Additionally, the API url path can be obtained after choosing the primary route and its subroutes which can then be copied and used here.

More information regarding EIA's API documentation can be found at `https://www.eia.gov/opendata/documentation.php`.

**Goal** - Combine the 2 different sub-routes for Electricity dating back from 2012 to 2022 to get a data report to help with the Machine Learning (ML) process to determine the most sustainable type of fuel category for electricity generation by:<br/>
* Categorizing fuel types into the following bin:
    * `Fossil fuels` - anthracite coal, bituminous coal, bituminous coal and syntehtic coal, 'coal, excluding waste coal', distillate fuel oil, fossil fuels, ignite coal, natural gas, natural gas & other gases, other gases, petroleum, petroleum coke, petroleum liquids, refined coal, residual fuel oil, subbituminous coal, lignite coal
        * From Emission data - coal, natural gas, petroleum
    * `Renewables` - biogenic municipal solid waste, biomass, conventional hydroelectric, estimated small scale solar photovoltaic, estimated total solar, estimated total solar photovoltaic, geothermal, hydro-electric pumped storage, landfill gas, municiapl landfill gas, offshore wind turbine, onshore wind turbine, renewable, renewable waste products, solar, solar photovoltaic, solar thermal, waste coal, waste oil and other oils, wind, wood and wood wastes, other renewables
        * From Emission data - total
    * `Others` - nuclear, other (sources not specified by EIA)

Binning information is based on sources from `https://www.eia.gov/tools/faqs/faq.php?id=427&t=3`, `https://www.eia.gov/electricity/data/browser/`, and `https://www.eia.gov/dnav/pet/TblDefs/pet_cons_821dst_tbldef2.asp`.

In [1]:
# Import dependencies
from config import api_key
import json
import requests
import pandas as pd

### Functions

In [2]:
def request_to_df(url, api_key, years = []):
    '''Function to request data from target API by looping through the years provided to return as a list before combining into one DataFrame'''
    data = []
    
    for year in years:
        api_path = url.replace('||KEY||', api_key).replace('||START||', year).replace('||END||', year)
        
        # Send the request
        response = requests.get(api_path).json()
        
        # Verify the response and raise error if bad response returned otherwise pass it into a DataFrame before returning it
        if (not 'warning' in response.keys()) and (not 'error' in response.keys()) :
            data += response['response']['data']
        else:
            raise Exception('Bad request submitted or no response received from the source API, verify that the url and/or offset provided is correct')
    
    df = pd.DataFrame(data)
    return df

def category_bin(df, check_col, list_to_bin, bin_name, new_col = ''):
    '''
        Function to create a bin category for a DataFrame based on the provided list then replace existing value with a bin category.
        If new_col is provided, a new column will be created for the binned category
    '''
    tmp_df = df.copy()
    
    for item in list_to_bin:
        if new_col == '' or new_col.isspace():
            tmp_df[check_col] = tmp_df[check_col].replace(item, bin_name)
        else:
            tmp_df.loc[tmp_df[check_col] == item, new_col] = bin_name
    
    return tmp_df

def fix_nan(df, col, fill_value = 0, to_type = 'float'):
    '''Function to fill the selected column's NaN value with the provided value and change it type'''
    tmp_df = df.copy()
    tmp_df[col] = tmp_df[col].fillna(fill_value)
    tmp_df[col] = tmp_df[col].astype(to_type)
    
    return tmp_df

### Extract

In [3]:
# Set years for the API to go through
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

# Set up the paths for retrieving the data into DataFrames with ||START|| and ||END|| for start and end parameter
epo_url = 'https://api.eia.gov/v2/electricity/electric-power-operational-data/data/?api_key=||KEY||&frequency=annual&data[0]=ash-content&data[1]=consumption-for-eg&data[2]=consumption-for-eg-btu&data[3]=consumption-uto&data[4]=consumption-uto-btu&data[5]=cost&data[6]=cost-per-btu&data[7]=generation&data[8]=heat-content&data[9]=receipts&data[10]=receipts-btu&data[11]=stocks&data[12]=sulfur-content&data[13]=total-consumption&data[14]=total-consumption-btu&start=||START||&end=||END||&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
emission_url = 'https://api.eia.gov/v2/electricity/state-electricity-profiles/emissions-by-state-by-fuel/data/?api_key=||KEY||&frequency=annual&data[0]=co2-rate-lbs-mwh&data[1]=co2-thousand-metric-tons&data[2]=nox-rate-lbs-mwh&data[3]=nox-short-tons&data[4]=so2-rate-lbs-mwh&data[5]=so2-short-tons&start=||START||&end=||END||&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'

In [4]:
# Get the request and create the DataFrames
epo_raw_df = request_to_df(epo_url, api_key, years)
emission_raw_df = request_to_df(emission_url, api_key, years)

In [5]:
# Export out raw data to csv files
epo_raw_df.to_csv('../static/data/epo_2012_2022_raw.csv', index = False)
emission_raw_df.to_csv('../static/data/emission_2012_2022_raw.csv', index = False)

In [6]:
# Print out the keys for epo
epo_raw_df.keys()

Index(['period', 'location', 'stateDescription', 'sectorid',
       'sectorDescription', 'fueltypeid', 'fuelTypeDescription', 'ash-content',
       'ash-content-units', 'consumption-for-eg', 'consumption-for-eg-units',
       'consumption-for-eg-btu', 'consumption-for-eg-btu-units',
       'consumption-uto', 'consumption-uto-units', 'consumption-uto-btu',
       'consumption-uto-btu-units', 'cost', 'cost-units', 'cost-per-btu',
       'cost-per-btu-units', 'generation', 'generation-units', 'heat-content',
       'heat-content-units', 'receipts', 'receipts-units', 'receipts-btu',
       'receipts-btu-units', 'stocks', 'stocks-units', 'sulfur-content',
       'sulfur-content-units', 'total-consumption', 'total-consumption-units',
       'total-consumption-btu', 'total-consumption-btu-units'],
      dtype='object')

In [7]:
# Print out the types
epo_raw_df.dtypes

period                          object
location                        object
stateDescription                object
sectorid                        object
sectorDescription               object
fueltypeid                      object
fuelTypeDescription             object
ash-content                     object
ash-content-units               object
consumption-for-eg              object
consumption-for-eg-units        object
consumption-for-eg-btu          object
consumption-for-eg-btu-units    object
consumption-uto                 object
consumption-uto-units           object
consumption-uto-btu             object
consumption-uto-btu-units       object
cost                            object
cost-units                      object
cost-per-btu                    object
cost-per-btu-units              object
generation                      object
generation-units                object
heat-content                    object
heat-content-units              object
receipts                 

In [8]:
# Print out the keys for emission
emission_raw_df.keys()

Index(['period', 'stateid', 'stateDescription', 'fuelid', 'fuelDescription',
       'co2-rate-lbs-mwh', 'co2-thousand-metric-tons', 'nox-rate-lbs-mwh',
       'nox-short-tons', 'so2-rate-lbs-mwh', 'so2-short-tons',
       'co2-rate-lbs-mwh-units', 'co2-thousand-metric-tons-units',
       'nox-rate-lbs-mwh-units', 'nox-short-tons-units',
       'so2-rate-lbs-mwh-units', 'so2-short-tons-units'],
      dtype='object')

In [9]:
# Print out the types
emission_raw_df.dtypes

period                            object
stateid                           object
stateDescription                  object
fuelid                            object
fuelDescription                   object
co2-rate-lbs-mwh                  object
co2-thousand-metric-tons          object
nox-rate-lbs-mwh                  object
nox-short-tons                    object
so2-rate-lbs-mwh                  object
so2-short-tons                    object
co2-rate-lbs-mwh-units            object
co2-thousand-metric-tons-units    object
nox-rate-lbs-mwh-units            object
nox-short-tons-units              object
so2-rate-lbs-mwh-units            object
so2-short-tons-units              object
dtype: object

### Transform

In [10]:
# Create copies of the DataFrames
epo_cleaned_df = epo_raw_df.copy()
emission_cleaned_df = emission_raw_df.copy()

---
**epo_df**

In [11]:
# Drop 'location', 'stateDescription', 'sectorid', 'sectorDescription', 'fueltypeid', 'consumption-for-eg', 'consumption-for-eg-units'
# 'consumption-uto', 'consumption-uto-units', 'cost', 'cost-units', 'receipts', 'receipts-units', 'total-consumption', 'total-consumption-units' 
# 'stocks-units', 'stocks'
# columns from epo_cleaned_dfabs
epo_cleaned_df = epo_cleaned_df.drop(['location', 'sectorid', 'fueltypeid'], axis = 1)
print(epo_cleaned_df.columns)

Index(['period', 'stateDescription', 'sectorDescription',
       'fuelTypeDescription', 'ash-content', 'ash-content-units',
       'consumption-for-eg', 'consumption-for-eg-units',
       'consumption-for-eg-btu', 'consumption-for-eg-btu-units',
       'consumption-uto', 'consumption-uto-units', 'consumption-uto-btu',
       'consumption-uto-btu-units', 'cost', 'cost-units', 'cost-per-btu',
       'cost-per-btu-units', 'generation', 'generation-units', 'heat-content',
       'heat-content-units', 'receipts', 'receipts-units', 'receipts-btu',
       'receipts-btu-units', 'stocks', 'stocks-units', 'sulfur-content',
       'sulfur-content-units', 'total-consumption', 'total-consumption-units',
       'total-consumption-btu', 'total-consumption-btu-units'],
      dtype='object')


In [12]:
# Print out the value_counts() in fuelTypeDescription for epo_cleaned_df
print('epo', epo_cleaned_df['fuelTypeDescription'].value_counts())

epo fuelTypeDescription
biomass                                     4525
all fuels                                   2546
fossil fuels                                2366
natural gas & other gases                   2318
natural gas                                 2222
renewable                                   2205
petroleum                                   2190
all renewables                              2130
petroleum liquids                           2110
distillate fuel oil                         2097
renewable waste products                    1718
all coal products                           1613
coal, excluding waste coal                  1535
other                                       1418
conventional hydroelectric                  1374
solar photovoltaic                          1338
solar                                       1312
bituminous coal                             1269
municiapl landfill gas                      1256
bituminous coal and synthetic coal          1

In [13]:
# Remove all rows where fuelTypeDescription for epo_cleaned_df is 'all coal products', 'all fuels', or 'all renewables'
epo_cleaned_df = epo_cleaned_df[
    ~epo_cleaned_df['fuelTypeDescription'].isin([
        'all coal products', 'all fuels', 'all renewables'
    ])
]

# Remove all rows where stateDescription contains 'U.S. Total'
epo_cleaned_df = epo_cleaned_df[
    ~epo_cleaned_df['stateDescription'].isin(['U.S. Total'])
]

In [14]:
# Create bins based on fuel type description into 'Fossil Fuels', 'Renewables', and 'Others' for epo_cleaned_df under
# energySource
epo_ff_source = [
    'anthracite coal', 'bituminous coal', 'bituminous coal and synthetic coal', 'coal, excluding waste coal', 
    'distillate fuel oil', 'fossil fuels', 'ignite coal', 'natural gas', 'natural gas & other gases', 'other gases', 
    'petroleum', 'petroleum coke', 'petroleum liquids', 'refined coal', 'residual fuel oil', 'subbituminous coal', 'lignite coal'
]

epo_renew_source = [
    'biogenic municipal solid waste', 'biomass', 'conventional hydroelectric', 'estimated small scale solar photovoltaic', 
    'estimated total solar', 'estimated total solar photovoltaic', 'geothermal', 'hydro-electric pumped storage', 
    'landfill gas', 'municiapl landfill gas', 'offshore wind turbine', 'onshore wind turbine', 'renewable', 
    'renewable waste products', 'solar', 'solar photovoltaic', 'solar thermal', 'waste coal', 'waste oil and other oils', 
    'wind', 'wood and wood wastes', 'other renewables'
]

epo_oth_source = [item for item in epo_cleaned_df['fuelTypeDescription'].value_counts().index if (item not in epo_ff_source and item not in epo_renew_source)]

epo_cleaned_df = category_bin(epo_cleaned_df, 'fuelTypeDescription', epo_ff_source, 'fossil fuels', 'energySource')
epo_cleaned_df = category_bin(epo_cleaned_df, 'fuelTypeDescription', epo_renew_source, 'renewables', 'energySource')
epo_cleaned_df = category_bin(epo_cleaned_df, 'fuelTypeDescription', epo_oth_source, 'others', 'energySource')

C:\Users\khang\AppData\Local\Temp\ipykernel_45212\2673199919.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'fossil fuels' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmp_df.loc[tmp_df[check_col] == item, new_col] = bin_name


In [15]:
# Check the value_counts() again to make sure binning was done correctly
epo_cleaned_df['energySource'].value_counts()

energySource
renewables      24400
fossil fuels    20716
others           1954
Name: count, dtype: int64

In [16]:
# Fill in NaN columns as 0 then set these columns as float
# 'ash-content', 'consumption-for-eg-btu', 'consumption-uto-btu', 'cost-per-btu', 'generation', 'heat-content',
# 'receipts-btu', 'sulfur-content', 'total-consumption-btu'
#
# Other units (in case needed more features): 'consumption-for-eg', 'consumption-uto', 'cost', 'receipts', 'total-consumption',
# 'stocks'
epo_cols = [
    'ash-content', 'consumption-for-eg-btu', 'consumption-uto-btu', 'cost-per-btu', 'generation', 'heat-content',
    'receipts-btu', 'sulfur-content', 'total-consumption-btu', 'consumption-for-eg', 'consumption-uto', 'cost', 
    'receipts', 'total-consumption', 'stocks'
]

for col in epo_cols:
    epo_cleaned_df = fix_nan(epo_cleaned_df, epo_cols)
    


In [17]:
# Convert the period column to DateTime format
epo_cleaned_df['period'] = pd.to_datetime(epo_cleaned_df['period'], format='%Y')

In [18]:
# Check the column types
epo_cleaned_df.dtypes

period                          datetime64[ns]
stateDescription                        object
sectorDescription                       object
fuelTypeDescription                     object
ash-content                            float64
ash-content-units                       object
consumption-for-eg                     float64
consumption-for-eg-units                object
consumption-for-eg-btu                 float64
consumption-for-eg-btu-units            object
consumption-uto                        float64
consumption-uto-units                   object
consumption-uto-btu                    float64
consumption-uto-btu-units               object
cost                                   float64
cost-units                              object
cost-per-btu                           float64
cost-per-btu-units                      object
generation                             float64
generation-units                        object
heat-content                           float64
heat-content-

In [19]:
epo_cleaned_df.columns

Index(['period', 'stateDescription', 'sectorDescription',
       'fuelTypeDescription', 'ash-content', 'ash-content-units',
       'consumption-for-eg', 'consumption-for-eg-units',
       'consumption-for-eg-btu', 'consumption-for-eg-btu-units',
       'consumption-uto', 'consumption-uto-units', 'consumption-uto-btu',
       'consumption-uto-btu-units', 'cost', 'cost-units', 'cost-per-btu',
       'cost-per-btu-units', 'generation', 'generation-units', 'heat-content',
       'heat-content-units', 'receipts', 'receipts-units', 'receipts-btu',
       'receipts-btu-units', 'stocks', 'stocks-units', 'sulfur-content',
       'sulfur-content-units', 'total-consumption', 'total-consumption-units',
       'total-consumption-btu', 'total-consumption-btu-units', 'energySource'],
      dtype='object')

In [20]:
# Rearrange the energySource to be next to fuelTypeDescription
epo_cleaned_df = epo_cleaned_df[[
    'period', 'stateDescription', 'sectorDescription',
    'energySource', 'fuelTypeDescription', 'ash-content', 'ash-content-units',
    'consumption-for-eg', 'consumption-for-eg-units',
    'consumption-for-eg-btu', 'consumption-for-eg-btu-units',
    'consumption-uto', 'consumption-uto-units', 'consumption-uto-btu',
    'consumption-uto-btu-units', 'cost', 'cost-units', 'cost-per-btu',
    'cost-per-btu-units', 'generation', 'generation-units', 'heat-content',
    'heat-content-units', 'receipts', 'receipts-units', 'receipts-btu',
    'receipts-btu-units', 'stocks', 'stocks-units', 'sulfur-content',
    'sulfur-content-units', 'total-consumption', 'total-consumption-units',
    'total-consumption-btu', 'total-consumption-btu-units'
]]

# Export the cleanned DataFrame for epo_cleaned_df into csv
epo_cleaned_df.to_csv('../static/data/epo_2012_2022_cleaned.csv', index = False)

**emission_df**

In [21]:
# Drop columns 'stateid', 'stateDescription', 'fuelid', 'co2-rate-lbs-mwh', 'nox-rate-lbs-mwh', 'so2-rate-lbs-mwh',
# 'co2-rate-lbs-mwh-units', 'nox-rate-lbs-mwh-units', 'so2-rate-lbs-mwh-units' from emission_cleaned_df
emission_cleaned_df = emission_cleaned_df.drop([
    'stateid', 'fuelid', 'co2-rate-lbs-mwh', 'nox-rate-lbs-mwh', 'so2-rate-lbs-mwh',
    'co2-rate-lbs-mwh-units', 'nox-rate-lbs-mwh-units', 'so2-rate-lbs-mwh-units'
], axis = 1)
print(emission_cleaned_df.columns)

Index(['period', 'stateDescription', 'fuelDescription',
       'co2-thousand-metric-tons', 'nox-short-tons', 'so2-short-tons',
       'co2-thousand-metric-tons-units', 'nox-short-tons-units',
       'so2-short-tons-units'],
      dtype='object')


In [22]:
# Print out the unique values in fuelDescription for emission_cleaned_df
print('emission', emission_cleaned_df['fuelDescription'].value_counts())

emission fuelDescription
Total          572
Petroleum      567
Natural Gas    561
Other          561
Coal           529
Name: count, dtype: int64


In [23]:
# Create bins based on fuel type description into 'Fossil Fuels', 'Renewables', and 'Others' for emission_cleaned_df under
# energySource
emission_ff_source = ['Petroleum', 'Natural Gas', 'Coal']
emission_renew_source = ['Total']
emission_oth_source = ['Other']

emission_cleaned_df = category_bin(emission_cleaned_df, 'fuelDescription', emission_ff_source, 'fossil fuels', 'energySource')
emission_cleaned_df = category_bin(emission_cleaned_df, 'fuelDescription', emission_renew_source, 'renewables', 'energySource')
emission_cleaned_df = category_bin(emission_cleaned_df, 'fuelDescription', emission_oth_source, 'others', 'energySource')

C:\Users\khang\AppData\Local\Temp\ipykernel_45212\2673199919.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'fossil fuels' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tmp_df.loc[tmp_df[check_col] == item, new_col] = bin_name


In [24]:
# Check the value_counts() again to make sure binning was done correctly
emission_cleaned_df['energySource'].value_counts()

energySource
fossil fuels    1657
renewables       572
others           561
Name: count, dtype: int64

In [25]:
# Fill in NaN columns as 0 then set these columns as float
# 'co2-thousand-metric-tons', 'nox-short-tons', 'so2-short-tons'
emission_cols = ['co2-thousand-metric-tons', 'nox-short-tons', 'so2-short-tons']

for col in emission_cols:
    emission_cleaned_df = fix_nan(emission_cleaned_df, emission_cols)

In [26]:
# Print out the columns
emission_cleaned_df.columns

Index(['period', 'stateDescription', 'fuelDescription',
       'co2-thousand-metric-tons', 'nox-short-tons', 'so2-short-tons',
       'co2-thousand-metric-tons-units', 'nox-short-tons-units',
       'so2-short-tons-units', 'energySource'],
      dtype='object')

In [27]:
# Convert the period column to DateTime format
emission_cleaned_df['period'] = pd.to_datetime(emission_cleaned_df['period'], format='%Y')

In [28]:
# Check the column types
emission_cleaned_df.dtypes

period                            datetime64[ns]
stateDescription                          object
fuelDescription                           object
co2-thousand-metric-tons                 float64
nox-short-tons                           float64
so2-short-tons                           float64
co2-thousand-metric-tons-units            object
nox-short-tons-units                      object
so2-short-tons-units                      object
energySource                              object
dtype: object

In [29]:
# Rearrange the energySource to be next to fuelTypeDescription
emission_cleaned_df = emission_cleaned_df[[
    'period', 'stateDescription', 'energySource', 'fuelDescription',
    'co2-thousand-metric-tons', 'nox-short-tons', 'so2-short-tons',
    'co2-thousand-metric-tons-units', 'nox-short-tons-units',
    'so2-short-tons-units'
]]

# Export the cleanned DataFrame for epo_cleaned_df into csv
emission_cleaned_df.to_csv('../static/data/emission_2012_2022_cleaned.csv', index = False)

### Database Storing
---

In [30]:
# Import dependencies for handling the database
from os import path, remove
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

In [31]:
# Setup the db path
db_path = '../static/data/eia_electric.sqlite'

# Delete the existing database if it exists
if path.exists(db_path):
    remove(db_path)

In [32]:
# Setup the engine and connect the database
engine = create_engine(f'sqlite:///{db_path}')
conn = engine.connect()

In [33]:
# Create session for querying later to verify tables have been created correctly
session = Session(bind = engine)

In [34]:
# Append the epo_cleaned_df to the database created
epo_cleaned_df.to_sql(name = 'epo', con = engine, if_exists = 'replace', index = False)

47070

In [35]:
session.execute(text('SELECT * from epo')).fetchone()

('2012-01-01 00:00:00.000000', 'West North Central', 'Electric Power Sector Non-CHP', 'fossil fuels', 'bituminous coal and synthetic coal', 9.55, 'percent', 1057.505, 'thousand short tons', 22.17242, 'million MMBtu', 0.0, 'thousand short tons', 0.0, 'million MMBtu', 0.0, 'dollars per short tons', 0.0, 'dollars per million Btu', 1940.31236, 'thousand megawatthours', 20.9667, 'Btu per short tons', 1019.951, 'thousand short tons', 22552.23513, 'billion Btu', 0.0, 'thousand short tons', 2.98, 'percent', 1057.505, 'thousand short tons', 22.17242, 'million MMBtu')

In [36]:
# Append the emission_cleaned_df to the database created
emission_cleaned_df.to_sql(name = 'emission', con = engine, if_exists = 'replace', index = False)

2790

In [37]:
session.execute(text('SELECT * from emission')).fetchone()

('2012-01-01 00:00:00.000000', 'Utah', 'fossil fuels', 'Natural Gas', 3113.0, 2126.0, 17.0, 'thousand metric tons', 'short tons', 'short tons')

In [38]:
# Close out of the session and engine
session.close()
engine.dispose()